## Reading 6-1 - Void Pointers

### Revisiting our Memory Metaphor

Up to this point in the course, everything you have done has been <b>static</b>, meaning that you have known the exact size of data elements when you began the program.

Since we've only used our working memory (registers) and static memory (hippocampus), we've only been able to "cram" information. 

<center><img src="https://github.com/mmorri22/su23-cse20332/blob/main/readings/reading02/Parable%20of%20the%20Choir.jpg?raw=true" alt="Memory Metaphor" width="500" height="600"></center>

<p> </p>
This approach has significant limitations, as we need to be able to store information <b>in real time</b>. Recall that - once you had "trained" yourself to remember large pieces of information - you could perform that task efficiently and reliably. You were actually using a different part of the brain, your neocortex.<p> </p>

We will call memory that we have to "train" - which is stored as we run the program - <b>dynamic memory</b>. And the computer's equivalent to the neocortex is called the <b>data heap</b>.

### Expanding our Memory Metaphor - Void Pointers

Remember that we remember by building links of brain neurons. But we can remember almost anything! Words, images, sounds, smells, tastes! Our neurons can be trained to remember any type of information we can remember. But they are <b>neurons first</b>.

Likewise, a computer can be trained to dynamically store any type of information it can store, but the memory is <b>binary bits first</b>.

Consider the movie <a href="https://www.netflix.com/title/80196789">Bird Box</a>. For those of you who have never seen Birdbox, the premise is that Earth has been invaded and that if a person sees the creatures, they will be lulled to their deaths. As a result, survivors learn how to navigate the world with blind folds, and have to make guesses as to what they are encountering. 

<center><img src="https://github.com/mmorri22/su23-cse20332/blob/main/readings/reading06/Reading%206-1.png?raw=true" alt="Bird Box" width="250" height="300"></center>

A <b>void pointer</b> is like programming with a blind fold: You are just allocating and pointing to memory. You have not yet defined the structure's data type.

We will use <b>void pointers</b> to point to chunks of binary memory that we allocate on the <b>Data Heap</b> 

### Creating a Void Pointer 

First, we need to use a new library, the C Standard Library Header File (<code>stdlib.h</code>). This library contains the functions we will need to write memory to the heap.

    #include <stdio.h>
    #include <stdlib.h>
    
In <code>main</code>, we will use a <code>long unsigned int</code> to store the number of <b>bytes</b> we plan to store.

    long unsigned int array_len = 13;
    
We will allocate memory of the number of bytes with a void pointer using a function called <code>malloc</code>, which takes in the number of bytes and returns a <code>void *</code>

    void* hello = malloc( array_len );

The <code>-Wconversion</code> Production Quality Compilation flag will strictly enforce that inputs to <code>malloc</code> are of type <code>long unsigned int</code>.
    
> <b>Test Yourself!</b> - Based on your understand of the values of signed and unsigned ints, why would we prefer to use an unsigned integer to represent the amount of memory we plan to allocate, as opposed to a signed int? And why would we prefer to use a <code>long unsigned int</code> as opposed to an <code>unsigned int</code>?

Now, we will print out the locations of every memory element we know. <code>void *</code> is just like the <code>int *</code> we learned about earlier. Recall that a pointer has three main elements:
<ul>
    <li>An initial location (origin)</li>
        <ul><li>Typically the address of a register</li></ul>
    <li>A destination</li>
        <ul><li>Stored in the origin register</li></ul>
    <li>A value</li>
        <ul><li>Information at that destination</li></ul>
</ul>

The difference is that void pointers do not have a value at the destination yet. We will cover how to store information at the destination in Reading 6.2.

    fprintf( stdout, "Location of array_len is %p\n", &array_len );
    fprintf( stdout, "Location of &hello is %p\n", &hello );
    fprintf( stdout, "Location of hello is %p\n", hello );
    
Based on the code at <a href = "">void_1.c</a>, here is a sample output run.

    > gcc -std=c11 -Wall -Wextra -Wconversion -Werror void_1.c -o void_1
    > ./void_1
    Location of array_len is 0x7ffc1c7a2088
    Location of &hello is 0x7ffc1c7a2080
    Location of hello is 0x1408010

> <b>Test Yourself!</b> - Why do you think the hexadecimal address for <code>hello</code> is so much shorter than <code>&array_len</code> or <code>&hello</code>?

It turns out that the address is <i>not</i> shorter at all. The actual address is <code>0x000001408010</code>!

The reason is that the memory addresses are commonly arranged such that Data Heap is closer to address 0x0, and the Stack and Registers are closer to the end of the memory.

<center><img src="https://github.com/mmorri22/su23-cse20332/blob/main/readings/reading06/Reading%206-2.png?raw=true" alt="First Memory Allocation" width="500" height="500"></center>


### Memory Leaks in Dynamically Allocated Memory

Before we can start assigning data types to our dynamic memory, we must learn the importance of <i>freeing</i> our memory. Whenever you allocate dynamic memory on the Data Heap, you must free that memory from the program.

It turns out we can implement another command line directive called <b>valgrind</b>. <a href = " https://valgrind.org/docs/manual/quick-start.html">Valgrind</a> allows us to determine if there is any leaked memory in our system, meaning we have not sufficiently freed all of our dynamic memory.

Consider the <code>void_1.c</code> that we ran above. I did not make any function call that freed the memory. Let's see what happens when I run the following command:
    
<code><font style="background-color:powderblue;">> gcc -std=c11 -Wall -Wextra -Wconversion -Werror void_1.c -o void_1</font> </code><br>
<code><font style="background-color:powderblue;">> valgrind --leak-check=full ./void_1</font></code><br>
<code>==16186== Memcheck, a memory error detector</code><br>
<code>==16186== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.</code><br>
<code>==16186== Using Valgrind-3.14.0 and LibVEX; rerun with -h for copyright info</code><br>
<code>==16186== Command: ./void_1</code><br>
<code>==16186==</code><br>
<code><font color="white" style="background-color:green;">Location of array_len is 0x1ffeffffb8</font></code><br>
<code><font color="white" style="background-color:green;">Location of &hello is 0x1ffeffffb0</font></code><br>
<code><font color="white" style="background-color:green;">Location of hello is 0x5205040</font></code><br>
<code>==16186== </code><br>
<code><font style="background-color:red;">==16186== HEAP SUMMARY:</font></code><br>
<code><font style="background-color:red;">==16186==     in use at exit: 13 bytes in 1 blocks</font></code><br>
<code><font style="background-color:red;">==16186==   total heap usage: 1 allocs, 0 frees, 13 bytes allocated</font></code><br>
<code>==16186==</code><br>
<code><font style="background-color:red;">==16186== 13 bytes in 1 blocks are definitely lost in loss record 1 of 1</font></code><br>
<code>==16186==    at 0x4C29E63: malloc (vg_replace_malloc.c:309)</code><br>
<code>==16186==    by 0x4005D8: main (in /escnfs/home/mmorri22/su23-cse20232/readings/reading06/void_1)</code><br>
<code>==16186==</code><br>
<code>==16186== LEAK SUMMARY:</code><br>
<code><font style="background-color:red;">==16186==    definitely lost: 13 bytes in 1 blocks</a></code><br>
<code>==16186==    indirectly lost: 0 bytes in 0 blocks</code><br>
<code>==16186==      possibly lost: 0 bytes in 0 blocks</code><br>
<code>==16186==    still reachable: 0 bytes in 0 blocks</code><br>
<code>==16186==         suppressed: 0 bytes in 0 blocks</code><br>
<code>==16186==</code><br>
<code>==16186== For counts of detected and suppressed errors, rerun with: -v</code><br>
<code><font style="background-color:red;">==16186== ERROR SUMMARY: 1 errors from 1 contexts (suppressed: 0 from 0)</font></code><p></p>

You can see that we performed one <code>malloc</code> (and no <code>frees</code>). We allocated 13 bytes with <code>malloc</code>, and that we never freed it, so we lost 13 bytes!

### Freeing Memory in Dynamic Memory

When you are finished using with dynamic memory, pass the <code>void *</code> variable to the <code>free</code> function.

    free( hello );
    
> <b>Remember this rule!</b> - Whenever you allocate memory, the first think you should do is write the code for when you free it.
    
The code for <a href = "https://raw.githubusercontent.com/mmorri22/su23-cse20332/main/readings/reading06/void_1_free.c">void_1_free.c</a> is the same as void_1.c, with the exception of a <code>free</code> call at the end.

<code><font style="background-color:powderblue;">> gcc -std=c11 -Wall -Wextra -Wconversion -Werror void_1_free.c -o void_1_free</font> </code><br>
<code><font style="background-color:powderblue;">> valgrind --leak-check=full ./void_1_free</font></code><br>
<code>==16186== Memcheck, a memory error detector</code><br>
<code>==16186== Copyright (C) 2002-2017, and GNU GPL'd, by Julian Seward et al.</code><br>
<code>==16186== Using Valgrind-3.14.0 and LibVEX; rerun with -h for copyright info</code><br>
<code>==16186== Command: ./void_1_free</code><br>
<code>==16186==</code><br>
<code><font color="white" style="background-color:green;">Location of array_len is 0x1ffeffffb8</font></code><br>
<code><font color="white" style="background-color:green;">Location of &hello is 0x1ffeffffb0</font></code><br>
<code><font color="white" style="background-color:green;">Location of hello is 0x5205040</font></code><br>
<code>==16186== </code><br>
<code><font style="background-color:cyan;">==16186== HEAP SUMMARY:</font></code><br>
<code><font style="background-color:cyan;">==16186==     in use at exit: 0 bytes in 0 blocks</font></code><br>
<code><font style="background-color:cyan;">==16186==   total heap usage: 1 allocs, 1 frees, 13 bytes allocated</font></code><br>
<code>==16186==</code><br>
<code><font style="background-color:cyan;">==16186== 13 bytes in 1 blocks are definitely lost in loss record 1 of 1</font></code><br>
<code>==17211==</code><br>
<code><font style="background-color:cyan;">==17211== All heap blocks were freed -- no leaks are possible</font></code><br>
<code>==17211==</code><br>
<code>==16186== For counts of detected and suppressed errors, rerun with: -v</code><br>
<code><font style="background-color:cyan;">==16186== ERROR SUMMARY: 0 errors from 0 contexts (suppressed: 0 from 0)</font></code><p></p>

You can see that we performed one <code>malloc</code> (and no <code>frees</code>). We allocated 13 bytes with <code>malloc</code>, and that we never freed it, so we lost 13 bytes!

### <font color = "red">Class Introduction Question #1 - Describe the differences between registers, static memory (on the Stack), and dynamic memory (on the Heap)</a>

### <font color = "red">Class Introduction Question #2 - What are void pointers, and how do we use them to dynamically allocate memory?</a>

### <font color = "red">Class Introduction Question #3 - What is the major benefit of using valgrind to run our programs as opposed to simple compilation?</a>

### The next reading for this lecture is <a href = "https://github.com/mmorri22/su23-cse20332/blob/main/readings/Reading%206-2.ipynb">Reading 6-2 - Pointer Casting and Arithmetic</a>